In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import pickle # Thư viện để lưu model

In [2]:
# Đọc dữ liệu
df_true = pd.read_csv('data/True.csv')
df_fake = pd.read_csv('data/Fake.csv')

# Gán nhãn: 1 cho tin thật, 0 cho tin giả
df_true['label'] = 1
df_fake['label'] = 0

# Gộp 2 dataframe, chỉ lấy cột text và label
df = pd.concat([df_true, df_fake])[['text', 'label']]

# Xáo trộn dữ liệu để đảm bảo tính ngẫu nhiên
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

print("Dữ liệu sau khi gộp:")
df.head()

Dữ liệu sau khi gộp:


,text,label
0,"Donald Trump s White House is in chaos, and th...",0
1,Now that Donald Trump is the presumptive GOP n...,0
2,Mike Pence is a huge homophobe. He supports ex...,0
3,SAN FRANCISCO (Reuters) - California Attorney ...,1
4,Twisted reasoning is all that comes from Pelos...,0


In [3]:
# Hàm làm sạch văn bản
def preprocess_text(text):
    text = text.lower() # Chuyển thành chữ thường
    text = re.sub(r'\s+', ' ', text) # Loại bỏ khoảng trắng thừa
    text = re.sub(r'\[.*?\]', '', text) # Loại bỏ các chuỗi như [Reuters]
    text = re.sub(r'<.*?>+', '', text) # Loại bỏ tag html
    text = re.sub(r'\w*\d\w*', '', text) # Loại bỏ từ chứa số
    return text

df['text'] = df['text'].apply(preprocess_text)

In [4]:
# Phân chia tập train và test
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Khởi tạo TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)

# Fit và transform tập train, chỉ transform tập test
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)

In [5]:
# Khởi tạo mô hình Passive Aggressive Classifier
pac = PassiveAggressiveClassifier(max_iter=50)
pac.fit(tfidf_train, y_train)

# Dự đoán trên tập test và tính độ chính xác
y_pred = pac.predict(tfidf_test)
score = accuracy_score(y_test, y_pred)

print(f"Độ chính xác của mô hình: {score*100:.2f}%")
print("Ma trận nhầm lẫn (Confusion Matrix):")
print(confusion_matrix(y_test, y_pred))

Độ chính xác của mô hình: 99.38%
Ma trận nhầm lẫn (Confusion Matrix):
[[4635   34]
 [  22 4289]]


In [6]:
# Tạo thư mục để lưu model nếu chưa có
import os
if not os.path.exists('saved_model'):
    os.makedirs('saved_model')

# Lưu model đã huấn luyện
with open('saved_model/model.pkl', 'wb') as f:
    pickle.dump(pac, f)

# Lưu vectorizer
with open('saved_model/vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)

print("Đã lưu model và vectorizer vào thư mục 'saved_model'")

Đã lưu model và vectorizer vào thư mục 'saved_model'
